In [20]:
# --------------------------------------------------------------------
# -- IMPORT PACKAGES
# --------------------------------------------------------------------

#from openslide import open_slide
import openslide
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from openslide.deepzoom import DeepZoomGenerator
import os
import imageio
from PIL import Image
import datetime
import json
from shapely.geometry import Polygon
from descartes import PolygonPatch
import cv2
from tqdm import tqdm

main_directory = "C:/Users/Administrateur/ProjetM1" 


# ----------------------------------------------------------------------------
# -- MAIN LOOP : runs code for each WSI in the folder
# ----------------------------------------------------------------------------

# select WSI folder path
WSIfolder_path = main_directory + '/WSI'

# create WSI files list 
WSI_list = [WSI for WSI in os.listdir(WSIfolder_path) if os.path.isfile(os.path.join(WSIfolder_path, WSI))]
#print("liste des WSI :", WSI_list) 

for WSI in WSI_list:

    t0 = datetime.datetime.now()
    print("Start time:", t0)
    
    name_WSI, ext_WSI = os.path.splitext(WSI) #name_WSI = filename without ".tif" extension
    
    #find corresponding annotation file
    found = False
    cancerous = False
    
    # cancerous annotations folder
    annotation_folder_path = main_directory + '/annotations/cancerous'
    annot_list = [annot for annot in os.listdir(annotation_folder_path) if os.path.isfile(os.path.join(annotation_folder_path, annot))] 
    for annot in annot_list:
        name_annot, ext_annot = os.path.splitext(annot)
        if name_WSI == name_annot and ext_WSI != ext_annot:
            annotation_file = annot
            found = True
            cancerous = True
    
    # try normal annotations folder if not found
    if found == False:
        annotation_folder_path = main_directory + '/annotations/normal'
        annot_list = [annot for annot in os.listdir(annotation_folder_path) if os.path.isfile(os.path.join(annotation_folder_path, annot))] 
        for annot in annot_list:
            name_annot, ext_annot = os.path.splitext(annot)
            if name_WSI == name_annot and ext_WSI != ext_annot:
                annotation_file = annot
                found = True
                
    # no correponding annotation, we skip the WSI            
    if found == False:
        print("No annotations found : SKIPPING TO NEXT WSI") 
        continue  
    
    #create patient folder
    patient = name_WSI[:3] # patient id
    patient_path = main_directory + '/data_saved/' + patient
    if not os.path.exists(patient_path):
        os.mkdir(patient_path)
        print("The folder " + patient + " has been created.")
    
    #create slide folder in patient folder
    slide_path = patient_path + '/' + name_WSI    
    if not os.path.exists(slide_path):
        os.mkdir(slide_path)
        print("The sub-folder " + name_WSI + " of patient n° " + patient + " has been created.")      
    
    #create cancerous_patches and normal_patches
    if not os.path.exists(slide_path + '/cancerous_patches'): 
        os.mkdir(slide_path + '/cancerous_patches')
    if not os.path.exists(slide_path + '/normal_patches'):     
        os.mkdir(slide_path + '/normal_patches') 
    print("The normal and cancerous patches subfolders have been created.")
        
    # informations about the current iteration of the loop    
    print("selected image :", WSI)
    print("selected annotation :", annotation_file)
    if cancerous == True:
        print("L'annotation correspondante entoure les zones cancéreuses.")
    else:
        print("Si une annotation correspondante est trouvée, elle entoure les zones normales.")
    
    
    #load WSI
    slide = openslide.open_slide(main_directory + '/WSI/' + WSI)
    
    # --------------------------------------------------------------------
    # 1--  CREATION TISSUE MASK
    # --------------------------------------------------------------------

    # Dimensions de l'image
    width, height = slide.dimensions
    print("The dimensions of the " + WSI + " slide at level 0 are :\n Width : " + str(width) + "\n Height : " + str(height))

    # Niveaux de zoom disponibles
    levels = slide.level_count

    # Niveau de zoom souhaité
    level = 4 #3

    # Charger l'image à ce niveau
    image = slide.read_region((0, 0), level, slide.level_dimensions[level])

    # Convertir l'image en un tableau numpy
    image = np.array(image)

    # Convertir l'image en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Seuillage
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)  
    
    mask = gray < thresh
    mask_tissue = mask.astype(int)*255

    # --------------------------------------------------------------------
    # 2-- CREATION ANNOTATED AREA MASK
    # --------------------------------------------------------------------

    slide_dims = slide.level_dimensions[4]

    # Find path of annotations file
    name_annot, ext_annot = os.path.splitext(annotation_file)
    if cancerous == True:
        folder_path = '/annotations/cancerous/'
    else:
        folder_path = '/annotations/normal/'

    # load file
    try:
        with open(main_directory + folder_path + name_annot + '.geojson') as f:
            jsonData = json.load(f)
    except:
        print('Failed!')
        print('ERROR while opening the JSON file!!')
        print(all_masks_dict) 

    # Only proceed if jsonData is not empty
    if len(jsonData) != 0:
        l_jsonData = list(jsonData)
        slide_key = l_jsonData[0]  # We assume there is only one initial key
        annotation_key = l_jsonData[1]
        pointsValues = []
        for i in range(len(jsonData[slide_key])):
            for zone in jsonData[annotation_key]:
                xi = [((points[0])) for points in zone]
                yi = [((points[1])) for points in zone]
                zone_pts = np.column_stack((xi, yi))
                pointsValues.append(zone_pts)

        # Make sure that we can draw valid contours. Certain masks (not endo or epi) have only one point.
        if len(pointsValues) > 0:
            # Resize the points
            scale = 1/16
            resized_pts = []
            for c_pts in pointsValues:
                resized_pts.append(np.squeeze(c_pts))
            resized_pts = [(scale * _pts).astype(int) for _pts in resized_pts]

            mask_annotarea = np.zeros((slide_dims[1], slide_dims[0]), dtype=np.uint8)
            for c in range(len(pointsValues)):
                cv2.drawContours(mask_annotarea, resized_pts, c, (255, 255, 255), -1)  # '-1' is a flag to filled the contour

  
    
    # --------------------------------------------------------------------
    # 3-- LOAD NECESSARY FILES WITH OPENSLIDE
    # --------------------------------------------------------------------    
    
    # load tissue mask
    #mask_tissue_path = main_directory + '/masks/tissue/' + name_WSI + '_tissue_mask.tif'
    #mask_tissue = openslide.open_slide(mask_tissue_path)
    
    # load area mask
    #mask_annotarea_path = main_directory + '/masks/annotated_area/' + name_WSI + '_annotarea_mask.tif'
    #mask_annotarea = openslide.open_slide(mask_annotarea_path)
    
    
    
    # ----------------------------------------------------------------------------
    # 4-- PATCH EXTRACTION : tissue mask + cancer area mask + save
    # ----------------------------------------------------------------------------
    
    # main directory to save patches
    savingdir = main_directory + "/data_saved/" + patient + "/" + name_WSI

    #size of patches
    patch_size = (512, 512)

    # other parameters
    threshold = 204
    level0_dim = slide.level_dimensions[0]
    #test = 0

    # Read patch by patch according to coodinates (with WSI level 0 dimensions)
    for y in tqdm(range(0, level0_dim[1], patch_size[1]), desc="COLONNES"):
        for x in range(0, level0_dim[0], patch_size[0]):
            #remove borders
            if x == 0 or x == 512 or x ==1024 or x==level0_dim[0] or x == level0_dim[0]-patch_size[0] or x == level0_dim[0]-2*patch_size[0] or y == 0 or y == 512 or y ==1024 or y==level0_dim[1] or y == level0_dim[1]-patch_size[1] or y == level0_dim[1]-2*patch_size[1] :
                continue
            
            ###________________TISSUE MASK___________________

            # masking the image with tissue mask : go through mask (acccording to patch by patch coordinates)
            mask_patch = mask_tissue[int(y/16):int((y+patch_size[1])/16),int(x/16):int((x+patch_size[1])/16)]

            # calculate color average of the patch
            if mask_patch.any():
                average = mask_patch.mean()
            else:
                average = 0
            print(WSI +" Average pixel value for TISUE mask patch (%d,%d) is: %.2f" % (x, y, average))

            # if patch is a white area in mask_tissue ( = tissue)
            if average > threshold:

                ###________________CANCER AREA INSIDE MASK (cancerous annotations)___________________

                if cancerous == True: #selected annotations folder = cancerous then inside of annotated area are cancerous
                #inside white area = cancer
                
                    # masking the image with area mask 
                    mask_patch = mask_annotarea[int(y/16):int((y+patch_size[1])/16),int(x/16):int((x+patch_size[1])/16)]

                    # calculate color average of the patch
                    if mask_patch.any():
                        average = mask_patch.mean()
                    else:
                        average = 0
                    print(WSI +" Average pixel value for ANNOTATED AREA mask patch (%d,%d) is: %.2f" % (x, y, average))

                    # if patch is in a white area of mask_annotarea ( = cancerous)
                    if average > threshold:

                        # read corresponding region in WSI
                        patch = slide.read_region((x, y), 1, patch_size)

                        # saving WSI patch in CANCEROUS TISSUE directory
                        tile_dir = savingdir + "/cancerous_patches" 
                        tile_name = os.path.join(tile_dir, '%d_%d' % (x, y))
                        print(WSI +" Now saving tile CANCEROUS TISSUE DIRECTORY with title: ", tile_name)
                        patch.save(tile_name + ".png")


                    # else not in cancerous area
                    else: 

                        # read corresponding region in WSI
                        patch = slide.read_region((x, y), 1, patch_size)                      

                        # saving patch in NORMAL TISSUE directory and name
                        tile_dir = savingdir + "/normal_patches"
                        tile_name = os.path.join(tile_dir, '%d_%d' % (x, y))
                        print(WSI +" Now saving tile in NORMAL TISSUE DIRECTORY with title: ", tile_name)
                        patch.save(tile_name + ".png")



                ###________________CANCER AREA OUTSIDE MASK (normal inside annotations)___________________     

                if cancerous == False : #selected annotations folder = normal then inside of annotated area are normal 
                #inside white area = normal
                
                    # masking the image with area mask 
                    mask_patch = mask_annotarea[int(y/16):int((y+patch_size[1])/16),int(x/16):int((x+patch_size[1])/16)]
    
                    # calculate color average of the patch
                    if mask_patch.any():
                        average = mask_patch.mean()
                    else:
                        average = 0
                    print(WSI +" Average pixel value for ANNOTATED AREA mask patch (%d,%d) is: %.2f" % (x, y, average))

                    # if patch is in a white area of mask_annotarea ( = normal)
                    if average > threshold:

                        # read corresponding region in WSI
                        patch = slide.read_region((x, y), 1, patch_size)

                        # saving WSI patch in NORMAL TISSUE directory
                        tile_dir = savingdir + "/normal_patches"
                        tile_name = os.path.join(tile_dir, '%d_%d' % (x, y))
                        print(WSI + " Now saving tile NORMAL TISSUE DIRECTORY with title : ", tile_name)
                        patch.save(tile_name + ".png")


                    # else not in normal area
                    else: 

                        # read corresponding region in WSI
                        patch = slide.read_region((x, y), 1, patch_size)                      

                        # saving patch in CANCEROUS TISSUE directory and name
                        tile_dir = savingdir + "/cancerous_patches"
                        tile_name = os.path.join(tile_dir, '%d_%d' % (x, y))
                        print(WSI + " Now saving tile in CANCEROUS TISSUE DIRECTORY with title : ", tile_name)
                        patch.save(tile_name + ".png")
                
                    
    # closing 
    slide.close()

    # verification prints
    t1 = datetime.datetime.now()
    print("End time:", t1)
    print("Elapsed time for " + WSI + " is : ", t1-t0)
    print("\n\n")

Start time: 2023-03-30 14:58:09.340981
No annotations found : SKIPPING TO NEXT WSI
Start time: 2023-03-30 14:58:09.340981
The normal and cancerous patches subfolders have been created.
selected image : 011A.tif
selected annotation : 011A.geojson
L'annotation correspondante entoure les zones cancéreuses.
The dimensions of the 011A.tif slide at level 0 are :
 Width : 66832
 Height : 161152


COLONNES:   2%|▏         | 6/315 [00:00<00:07, 42.92it/s]

011A.tif Average pixel value for TISUE mask patch (1536,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (2048,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (2560,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (3072,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (3584,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (4096,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (4608,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (5120,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (5632,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (6144,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (6656,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (7168,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (7680,1536) is: 0.00
011A.tif Average pixel value for TISUE mask patch (8192,1536) is: 0.00
011A.t

011A.tif Average pixel value for TISUE mask patch (34816,3072) is: 206.69
011A.tif Average pixel value for ANNOTATED AREA mask patch (34816,3072) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\34816_3072
011A.tif Average pixel value for TISUE mask patch (35328,3072) is: 216.40
011A.tif Average pixel value for ANNOTATED AREA mask patch (35328,3072) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\35328_3072
011A.tif Average pixel value for TISUE mask patch (35840,3072) is: 192.25
011A.tif Average pixel value for TISUE mask patch (36352,3072) is: 195.73
011A.tif Average pixel value for TISUE mask patch (36864,3072) is: 190.00
011A.tif Average pixel value for TISUE mask patch (37376,3072) is: 147.92
011A.tif Average pixel value for TISUE mask patch (37888,3072) is: 11.46
011A.tif Average pixel 

011A.tif Average pixel value for TISUE mask patch (36352,3584) is: 193.24
011A.tif Average pixel value for TISUE mask patch (36864,3584) is: 190.25
011A.tif Average pixel value for TISUE mask patch (37376,3584) is: 173.57
011A.tif Average pixel value for TISUE mask patch (37888,3584) is: 162.36
011A.tif Average pixel value for TISUE mask patch (38400,3584) is: 52.29
011A.tif Average pixel value for TISUE mask patch (38912,3584) is: 9.96
011A.tif Average pixel value for TISUE mask patch (39424,3584) is: 0.00
011A.tif Average pixel value for TISUE mask patch (39936,3584) is: 0.00
011A.tif Average pixel value for TISUE mask patch (40448,3584) is: 0.00
011A.tif Average pixel value for TISUE mask patch (40960,3584) is: 0.00
011A.tif Average pixel value for TISUE mask patch (41472,3584) is: 0.00
011A.tif Average pixel value for TISUE mask patch (41984,3584) is: 0.00
011A.tif Average pixel value for TISUE mask patch (42496,3584) is: 0.00
011A.tif Average pixel value for TISUE mask patch (4300

...

011A.tif Average pixel value for TISUE mask patch (40960,92672) is: 204.20
011A.tif Average pixel value for ANNOTATED AREA mask patch (40960,92672) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\40960_92672
011A.tif Average pixel value for TISUE mask patch (41472,92672) is: 225.37
011A.tif Average pixel value for ANNOTATED AREA mask patch (41472,92672) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\41472_92672
011A.tif Average pixel value for TISUE mask patch (41984,92672) is: 238.32
011A.tif Average pixel value for ANNOTATED AREA mask patch (41984,92672) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\41984_92672
011A.tif Average pixel value for TISUE mask patch (42496,92672) is: 236.07
011A

011A.tif Average pixel value for TISUE mask patch (40960,93184) is: 233.09
011A.tif Average pixel value for ANNOTATED AREA mask patch (40960,93184) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\40960_93184
011A.tif Average pixel value for TISUE mask patch (41472,93184) is: 240.31
011A.tif Average pixel value for ANNOTATED AREA mask patch (41472,93184) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\41472_93184
011A.tif Average pixel value for TISUE mask patch (41984,93184) is: 246.78
011A.tif Average pixel value for ANNOTATED AREA mask patch (41984,93184) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\41984_93184
011A.tif Average pixel value for TISUE mask patch (42496,93184) is: 241.05
011A

COLONNES:  58%|█████▊    | 182/315 [00:46<00:34,  3.89it/s]

011A.tif Average pixel value for TISUE mask patch (43008,93184) is: 246.28
011A.tif Average pixel value for ANNOTATED AREA mask patch (43008,93184) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\43008_93184
011A.tif Average pixel value for TISUE mask patch (43520,93184) is: 242.05
011A.tif Average pixel value for ANNOTATED AREA mask patch (43520,93184) is: 255.00
011A.tif Now saving tile CANCEROUS TISSUE DIRECTORY with title:  C:/Users/Administrateur/ProjetM1/data_saved/011/011A/cancerous_patches\43520_93184


KeyboardInterrupt: 